In [38]:
def switch_left():
    ############## iframe으로 왼쪽 포커스 맞추기 ##############
    driver.switch_to.parent_frame()
    iframe = driver.find_element(By.XPATH,'//*[@id="searchIframe"]')
    driver.switch_to.frame(iframe)
    
def switch_right():
    ############## iframe으로 오른쪽 포커스 맞추기 ##############
    driver.switch_to.parent_frame()
    iframe = driver.find_element(By.XPATH,'//*[@id="entryIframe"]')
    driver.switch_to.frame(iframe)

In [39]:
class Colors:
    RED = '\033[91m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    BLUE = '\033[94m'
    MAGENTA = '\033[95m'
    CYAN = '\033[96m'
    RESET = '\033[0m'

# 데이터 크롤링

In [44]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from time import sleep
import random
import re
import pandas as pd
from selenium import webdriver
import sys

options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
options.add_argument('window-size=1380,900')
driver = webdriver.Chrome(options=options)

# 대기 시간
driver.implicitly_wait(time_to_wait=5)

# 반복 종료 조건
loop = True

# 네이버 지도에서 검색할 위치
search_location = "수성구 숙소"

# 네이버 지도 검색 페이지로 이동
driver.get(f"https://map.naver.com/v5/search/{search_location}")

# 맛집 이름과 평점 크롤링
lodging = []
while(True):
    switch_left()

    # 페이지 숫자를 초기에 체크 [ True / False ]
    # 이건 페이지 넘어갈때마다 계속 확인해줘야 함 (페이지 새로 로드 될때마다 버튼 상태 값이 바뀜)
    next_page = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div[2]/div[2]/a[2]').get_attribute('aria-disabled')
    
    if(next_page == 'true'):
        break

     #찾고자 하는 스크롤 가능한 요소
    scrollable_element = driver.find_element(By.CLASS_NAME, "Ryr1F")
    
    # 이전 높이와 새로운 높이 비교를 위해 초기화
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
    
    while True:
        # 스크롤 내리기
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_element)
        sleep(2)  # 로드 시간 확보
    
        # 새로운 높이 확인
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
    
        if new_height == last_height:
            print("스크롤 끝에 도달하거나 새로운 콘텐츠가 로드되지 않음")
            break
    
        last_height = new_height

    
    ############## 현재 page number 가져오기 - 1 페이지 ##############

    page_no = driver.find_element(By.XPATH,'//a[contains(@class, "mBN2s qxokY")]').text

    # 현재 페이지에 등록된 모든 가게 조회
    # 첫페이지 광고 2개 때문에 첫페이지는 앞 2개를 빼야함
    if(page_no == '1'):
        elemets = driver.find_elements(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]//li')[2:]
    else:
        elemets = driver.find_elements(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]//li')
    
    #현재가게 찾기(프린트 화면으로)
    print('현재 ' + '\033[95m' + str(page_no) + '\033[0m' + ' 페이지 / '+ '총 ' + '\033[95m' + str(len(elemets)) + '\033[0m' + '개의 가게를 찾았습니다.\n')

    for index, e in enumerate(elemets, start=1):
        final_element = e.find_element(By.CLASS_NAME,'ouxiq').find_element(By.XPATH, ".//a/div/div/span")

        print(str(index) + ". " + final_element.text)

    print(Colors.RED + "-"*50 + Colors.RESET)

    switch_left()

    sleep(3)

    for index, e in enumerate(elemets, start=1):
        store_name = '' # 가게 이름        
        rating = 0.0 # 평점        
        address = '' # 가게 주소
        phone_num = '' # 전화번호
        switch_left()

        # 순서대로 값을 하나씩 클릭
        e.find_element(By.CLASS_NAME,'ouxiq').find_element(By.XPATH, ".//a/div/div/span").click()
        
        sleep(2)

        switch_right()

        ################### 여기부터 크롤링 시작 ##################
        #식당이름
        title = driver.find_element(By.XPATH,'//div[@class="zD5Nm undefined"]')
        #가게정보
        store_info = title.find_elements(By.XPATH,'//div[@class="YouOG DZucB"]/div/span')
        # 카테고리
        category = title.find_element(By.XPATH,'.//div[1]/div[1]/span[2]').text
        # 리뷰
        review = title.find_elements(By.XPATH,'.//div[2]/span')
        # 가게 이름
        store_name = title.find_element(By.XPATH,'.//div[1]/div[1]/span[1]').text
         # 가게 전화번호
        try:
            phone_num = driver.find_element(By.XPATH,'//span[@class="xlx7Q"]').text
        except NoSuchElementException:
            phone_num = '없음'
        # 요소를 찾기 위한 XPath
        xpaths = ['//*[@id="ibu_1"]', '//*[@id="cp0_2"]','//*[@id="app-root"]/div/div/div/div[1]/div/div[1]/div/a/span','//*[@id="app-root"]/div/div/div/div[1]/div/a/img']               
        # 이미지 소스 저장 변수
        img_src = None
            
        #  XPath를 순차적으로 시도
        for xpath in xpaths:
            elements = driver.find_elements(By.XPATH, xpath)
            
            if elements:
                first_element = elements[0]  # 첫 번째 요소 선택
                
                if first_element.tag_name == 'img':  # 첫 번째 요소가 이미지인지 확인
                    img_src = first_element.get_attribute('src')
                    print("First image source found:", img_src)
                    break  # 이미지를 찾았으므로 반복 종료
                
                elif first_element.tag_name == 'video':  # 첫 번째 요소가 동영상인 경우
                    print("First element is a video. Searching for the third image...")
                    
                    # 세 번째 이미지 요소가 있는지 확인
                    img_elements = [el for el in elements if el.tag_name == 'img']
                    
                    if len(img_elements) >= 3:  # 세 번째 이미지가 있는지 확인
                        img_src = img_elements[2].get_attribute('src')  # 세 번째 이미지의 'src' 가져오기
                        print("Third image source found:", img_src)
                        break  # 세 번째 이미지를 찾았으므로 반복 종료
                    else:
                        img_src = first_element.get_attribute('src')
                        print("Less than three images found in the current XPath.")
            
            print(f"No valid image found at XPath: {xpath}, trying next XPath...")

        
        
        
        # 인덱스 변수 값
        _index = 1

        # 리뷰 ROW의 갯수가 3개 이상일 경우 [별점, 방문자 리뷰, 블로그 리뷰]
        if len(review) > 2:
            rating_xpath = f'.//div[2]/span[{_index}]'
            rating_element = title.find_element(By.XPATH, rating_xpath)
            rating = rating_element.text.replace("\n", " ")

            _index += 1
        
        # 가게 주소
        address = driver.find_element(By.XPATH,'//span[@class="LDgIH"]').text
 
        
        # 데이터 저장
        lodging.append({
            "store_name": store_name,
            "rating":rating,
            "category":category,
            "address": address,
            "phone_num": phone_num,
            "img":img_src
        })
    switch_left()
    break    
    # 페이지 다음 버튼이 활성화 상태일 경우 계속 진행
    if(next_page == 'false'):
        driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div[2]/div[2]/a[2]').click()
    # 아닐 경우 루프 정지
    else:
        loop = False

# 데이터 프레임으로 저장
lf = pd.DataFrame(lodging)     

스크롤 끝에 도달하거나 새로운 콘텐츠가 로드되지 않음
현재 1 페이지 / 총 48개의 가게를 찾았습니다.

1. 하늘아래수성별사랑캠프
2. 호텔 피에드 수성
3. 대구 그랜드호텔
4. 호텔라온제나
5. 진밭골야영장
6. ACT 관광호텔
7. 2월호텔 라이온즈파크점
8. 아리아나호텔
9. 예향아트하우스
10. 가우진 호텔
11. 수성레이크게스트하우스
12. 뉴골드장모텔
13. 폭스모텔
14. TOP365 모텔
15. 캐슬모텔
16. 숙박업
17. 자야모텔
18. 타임모텔
19. 오렌지모텔
20. 롯데모텔
21. 로잔모텔
22. J모텔
23. 에스모텔
24. 조이
25. 아이 모텔
26. 코리아나모텔
27. 드라마모텔
28. 에바스모텔
29. 뷰티크호텔
30. 대구 수성구 J
31. 히든파라다이스
32. 송림모텔
33. 첫눈부띠끄호텔
34. 프라임 모텔
35. 리첸시아모텔
36. 엘레강스
37. R2모텔
38. 원빈모텔
39. 엠투고모텔
40. 에르메스호텔
41. 다이아나모텔
42. 샤프모텔
43. 줄리엣모텔
44. 베르사체모텔
45. J(제이)
46. 에이치센트럴
47. 애플모텔
48. 엘리자호텔
--------------------------------------------------
No valid image found at XPath: //*[@id="ibu_1"], trying next XPath...
First image source found: https://search.pstatic.net/common/?autoRotate=true&type=w278_sharpen&src=https%3A%2F%2Fvideo-phinf.pstatic.net%2F20240824_111%2F1724493568666Dgn5K_JPEG%2FYAkC9iZ6eJ_03.jpg
First image source found: https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb

In [45]:
lf

,store_name,rating,category,address,phone_num,img
0,하늘아래수성별사랑캠프,0.0,"캠핑,야영장",대구 수성구 진밭길 420,0507-1437-1168,https://search.pstatic.net/common/?autoRotate=...
1,호텔 피에드 수성,0.0,3급,대구 수성구 알파시티1로4길 18 호텔 피에드 수성,0507-1448-2902,https://search.pstatic.net/common/?autoRotate=...
2,대구 그랜드호텔,별점 4.25,4급,대구 수성구 동대구로 305 그랜드관광호텔,053-742-0001,https://search.pstatic.net/common/?autoRotate=...
3,호텔라온제나,별점 4.39,4성급,대구 수성구 범어천로 73,053-718-7000,https://search.pstatic.net/common/?autoRotate=...
4,진밭골야영장,0.0,"캠핑,야영장",대구 수성구 진밭길 112,053-666-3477,https://search.pstatic.net/common/?autoRotate=...
5,ACT 관광호텔,0.0,3성급,대구 수성구 알파시티1로32길 28-17,0507-1384-0000,https://search.pstatic.net/common/?autoRotate=...
6,2월호텔 라이온즈파크점,0.0,3급,대구 수성구 알파시티1로4길 19-7 2월호텔 라이온즈파크점,053-792-2022,https://search.pstatic.net/common/?autoRotate=...
7,아리아나호텔,별점 4.38,3성급,대구 수성구 동대구로 27 아리아나호텔,0507-1428-7803,https://search.pstatic.net/common/?autoRotate=...
8,예향아트하우스,0.0,펜션,대구 수성구 고모로18길 22,010-3537-0719,https://search.pstatic.net/common/?autoRotate=...
9,가우진 호텔,0.0,모텔,대구 수성구 수성로 276,0507-1479-1158,None


In [46]:
# 데이터프레임을 CSV 파일로 저장
#csv_file_path = '수성구숙소.csv'
#lf.to_csv(csv_file_path, index=False)  # index=False로 인덱스를 제외하고 저장

In [51]:
lodf = pd.read_csv('수성구숙소.csv' , encoding= 'CP949')

In [52]:
lodf

,store_name,rating,category,address,phone_num,img
0,하늘아래수성별사랑캠프,0,"캠핑,야영장",대구 수성구 진밭길 420,0507-1437-1168,https://search.pstatic.net/common/?autoRotate=...
1,호텔 피에드 수성,0,3급,대구 수성구 알파시티1로4길 18 호텔 피에드 수성,0507-1448-2902,https://search.pstatic.net/common/?autoRotate=...
2,대구 그랜드호텔,별점 4.25,4급,대구 수성구 동대구로 305 그랜드관광호텔,053-742-0001,https://search.pstatic.net/common/?autoRotate=...
3,호텔라온제나,별점 4.39,4성급,대구 수성구 범어천로 73,053-718-7000,https://search.pstatic.net/common/?autoRotate=...
4,진밭골야영장,0,"캠핑,야영장",대구 수성구 진밭길 112,053-666-3477,https://search.pstatic.net/common/?autoRotate=...
5,ACT 관광호텔,0,3성급,대구 수성구 알파시티1로32길 28-17,0507-1384-0000,https://search.pstatic.net/common/?autoRotate=...
6,2월호텔 라이온즈파크점,0,3급,대구 수성구 알파시티1로4길 19-7 2월호텔 라이온즈파크점,053-792-2022,https://search.pstatic.net/common/?autoRotate=...
7,아리아나호텔,별점 4.38,3성급,대구 수성구 동대구로 27 아리아나호텔,0507-1428-7803,https://search.pstatic.net/common/?autoRotate=...
8,예향아트하우스,0,펜션,대구 수성구 고모로18길 22,010-3537-0719,https://search.pstatic.net/common/?autoRotate=...
9,가우진 호텔,0,모텔,대구 수성구 수성로 276,0507-1479-1158,https://search.pstatic.net/common/?src=https%3...


In [53]:
raddress = lodf['address']
raddress

0                        대구 수성구 진밭길 420
1          대구 수성구 알파시티1로4길 18 호텔 피에드 수성
2               대구 수성구 동대구로 305 그랜드관광호텔
3                        대구 수성구 범어천로 73
4                        대구 수성구 진밭길 112
5                대구 수성구 알파시티1로32길 28-17
6     대구 수성구 알파시티1로4길 19-7 2월호텔 라이온즈파크점
7                 대구 수성구 동대구로 27 아리아나호텔
8                      대구 수성구 고모로18길 22
9                        대구 수성구 수성로 276
10                  대구 수성구 용학로30길 12 3층
11                    대구 수성구 달구벌대로 2546
12                    대구 수성구 청수로 126-36
13            대구 수성구 청수로25길 25 TOP365모텔
14                       대구 수성구 동대구로 36
15                      대구 수성구 욱수천로 133
16                       대구 수성구 수성로 205
17                     대구 수성구 청수로25길 27
18                     대구 수성구 용학로25길 14
19                  대구 수성구 달구벌대로 2319-2
20                       대구 수성구 동대구로 70
21                        대구 수성구 수성로 18
22                    대구 수성구 세진로3길 51-1
23                    대구 수성구 동대구로25길 21
24                        대구 수성구 수성로 21


In [55]:
#쪼개기 활용화
#traddress = lodf[['address']].copy()
#traddress.head()

,address
0,대구 수성구 진밭길 420
1,대구 수성구 알파시티1로4길 18 호텔 피에드 수성
2,대구 수성구 동대구로 305 그랜드관광호텔
3,대구 수성구 범어천로 73
4,대구 수성구 진밭길 112


In [56]:
import re
# "층", "호" 및 주소의 끝에 있는 추가 정보 제거하고 숫자 번지수만 남기는 함수 정의
# 주소에서 숫자 번지수와 그 이전의 주소 부분만 유지하는 함수 정의
def clean_address(address):
    # 주소에서 숫자 번지수와 그 이전의 주소 부분만 유지하고 그 이후의 정보를 제거
    match = re.search(r'^.*?\d+(?:-\d+)?(?:\s|$)', address)
    if match:
        return match.group().strip()  # 추출된 주소 부분만 반환
    return address.strip()

In [57]:
#traddress['address'] = traddress['address'].apply(clean_address)

In [59]:
#traddress

,address
0,대구 수성구 진밭길 420
1,대구 수성구 알파시티1로4길 18
2,대구 수성구 동대구로 305
3,대구 수성구 범어천로 73
4,대구 수성구 진밭길 112
5,대구 수성구 알파시티1로32길 28-17
6,대구 수성구 알파시티1로4길 19-7
7,대구 수성구 동대구로 27
8,대구 수성구 고모로18길 22
9,대구 수성구 수성로 276


In [60]:
raddress = lodf['address'].apply(clean_address)

In [61]:
raddress

0             대구 수성구 진밭길 420
1         대구 수성구 알파시티1로4길 18
2            대구 수성구 동대구로 305
3             대구 수성구 범어천로 73
4             대구 수성구 진밭길 112
5     대구 수성구 알파시티1로32길 28-17
6       대구 수성구 알파시티1로4길 19-7
7             대구 수성구 동대구로 27
8           대구 수성구 고모로18길 22
9             대구 수성구 수성로 276
10          대구 수성구 용학로30길 12
11         대구 수성구 달구벌대로 2546
12         대구 수성구 청수로 126-36
13          대구 수성구 청수로25길 25
14            대구 수성구 동대구로 36
15           대구 수성구 욱수천로 133
16            대구 수성구 수성로 205
17          대구 수성구 청수로25길 27
18          대구 수성구 용학로25길 14
19       대구 수성구 달구벌대로 2319-2
20            대구 수성구 동대구로 70
21             대구 수성구 수성로 18
22         대구 수성구 세진로3길 51-1
23         대구 수성구 동대구로25길 21
24             대구 수성구 수성로 21
25         대구 수성구 동대구로25길 34
26          대구 수성구 수성로 258-1
27            대구 수성구 화랑로 132
28           대구 수성구 동대구로 165
29             대구 수성구 수성로 18
30             대구 수성구 용학로 13
31              대구 수성구 수성로 4
32           대구 수성구 두산동 81-5
33           대구 수성구 동대구로길 25
34           대

# 위도 경도 값으로 바꿔주기

In [62]:
####### 도로명주소 위도 경도 값으로 바꿔주기 ########
from geopy.geocoders import Nominatim
geo_local = Nominatim(user_agent='South Korea')

# 위도, 경도 반환하는 함수
def geocoding(raddress):
    try:
        geo = geo_local.geocode(raddress)
        x_y = [geo.latitude, geo.longitude]
        return x_y

    except:
        return [0,0]
#####주소를 위,경도 값으로 변환하기 #####
latitude = []
longitude =[]

for i in raddress:
    latitude.append(geocoding(i)[0])
    longitude.append(geocoding(i)[1])

#####Dataframe만들기######
new = pd.DataFrame({'store_name': lodf['store_name'],'rating':lodf['rating'],'category':lodf['category'],'phone_num':lodf['phone_num'],'img':lodf['img'],'address':raddress,'latitude':latitude,'longitude':longitude})
new.head()

,store_name,rating,category,phone_num,img,address,latitude,longitude
0,하늘아래수성별사랑캠프,0,"캠핑,야영장",0507-1437-1168,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 진밭길 420,35.817108,128.655764
1,호텔 피에드 수성,0,3급,0507-1448-2902,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 알파시티1로4길 18,35.839349,128.688500
2,대구 그랜드호텔,별점 4.25,4급,053-742-0001,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 동대구로 305,35.857751,128.625358
3,호텔라온제나,별점 4.39,4성급,053-718-7000,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 범어천로 73,35.861821,128.621766
4,진밭골야영장,0,"캠핑,야영장",053-666-3477,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 진밭길 112,35.817108,128.655764


In [63]:
new

,store_name,rating,category,phone_num,img,address,latitude,longitude
0,하늘아래수성별사랑캠프,0,"캠핑,야영장",0507-1437-1168,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 진밭길 420,35.817108,128.655764
1,호텔 피에드 수성,0,3급,0507-1448-2902,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 알파시티1로4길 18,35.839349,128.688500
2,대구 그랜드호텔,별점 4.25,4급,053-742-0001,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 동대구로 305,35.857751,128.625358
3,호텔라온제나,별점 4.39,4성급,053-718-7000,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 범어천로 73,35.861821,128.621766
4,진밭골야영장,0,"캠핑,야영장",053-666-3477,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 진밭길 112,35.817108,128.655764
5,ACT 관광호텔,0,3성급,0507-1384-0000,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 알파시티1로32길 28-17,35.836909,128.682084
6,2월호텔 라이온즈파크점,0,3급,053-792-2022,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 알파시티1로4길 19-7,35.839349,128.688500
7,아리아나호텔,별점 4.38,3성급,0507-1428-7803,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 동대구로 27,35.844965,128.625647
8,예향아트하우스,0,펜션,010-3537-0719,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 고모로18길 22,35.850541,128.687437
9,가우진 호텔,0,모텔,0507-1479-1158,https://search.pstatic.net/common/?src=https%3...,대구 수성구 수성로 276,35.861422,128.614265


In [66]:
new['rating'] = new['rating'].replace('0','미등록')

In [392]:
#new.rename(columns={'위도':'latitude','경도':'longitude'}, inplace=True)

In [68]:
# 데이터프레임을 CSV 파일로 저장
csv_file_path = '수성구숙소 이미지 경도버전.csv'
new.to_csv(csv_file_path, index=False)  # index=False로 인덱스를 제외하고 저장

# DB 적재

In [69]:
import cx_Oracle

In [70]:
# 라이브러리 임포트
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData

In [71]:
# 데이터 베이스 연결 정보 설정
db_username = 'base_man'
db_password = '1111'
db_host = '192.168.0.29' # 다른 IP 주소
db_port = '1521' #Oracle의 기본포트
service_name = 'XE'

# Oracle 데이터베이스 연결 문자열 생성
dsn_tns = cx_Oracle.makedsn(db_host, db_port, service_name=service_name)

# 데이터베이스 연결
connection = cx_Oracle.connect(user=db_username, password=db_password, dsn=dsn_tns)

# 커서 생성
cursor = connection.cursor()

In [72]:
# 테이블 생성 쿼리
create_table_query = '''
create table lodging(
    store_name varchar(50),
    rating varchar(30),
    category varchar(30),
    address varchar(100),
    phone_num varchar(50),
    latitude number(9, 6),
    longitude number(9, 6),
    img varchar(500)
)
'''
# 쿼리 실행
cursor.execute(create_table_query)

#변경 사항 커밋
connection.commit()

#커서와 연결 종료
cursor.close()
connection.close()

#성공 알람
print("Table created successfully!!")

Table created successfully!!


In [59]:
# #테이블 삭제 쿼리(필요시 사용할것)
# drop_restaurants = "DROP TABLE lodging"
# #쿼리 실행
# cursor.execute(drop_restaurants)

# #변경 사항 커밋
# connection.commit()

# #커서와 연결 종료
# cursor.close()
# connection.close()

# #성공 알람
# print("Table drop successfully!!")

Table drop successfully!!


# 식당 정보를 DB에 적재

In [73]:
from sqlalchemy import create_engine
from sqlalchemy.dialects.oracle import NUMBER, VARCHAR2

# 데이터 베이스 연결 정보 설정
db_username = 'base_man'
db_password = '1111'
db_host = '192.168.0.29' # 다른 IP 주소
db_port = '1521' #Oracle의 기본포트
service_name = 'XE'

engine = create_engine(f'oracle+cx_oracle://{db_username}:{db_password}@{db_host}:{db_port}/?service_name={service_name}')
new.to_sql(
    'lodging', 
    con=engine, 
    if_exists='replace', 
    index=False, 
    dtype={
        'store_name': VARCHAR2(50),
        'rating': VARCHAR2(30),
        'category': VARCHAR2(30),
        'address' :VARCHAR2(100),
        'phone_num': VARCHAR2(50),
        'img':VARCHAR2(500),
        'latitude': NUMBER(precision=9, scale=6), #위도
        'longitude': NUMBER(precision=9, scale=6)  #경도
    }
)
print("Table insert successfully")

Table insert successfully


In [75]:
#db에서 가져옴
query = "select * from lodging"
loddf = pd.read_sql(query, con=engine)

In [79]:
loddf

,store_name,rating,category,phone_num,img,address,latitude,longitude
0,엘리자호텔,미등록,모텔,053-761-3234,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 동대구로 121,35.861471,128.627109
1,하늘아래수성별사랑캠프,미등록,"캠핑,야영장",0507-1437-1168,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 진밭길 420,35.817108,128.655764
2,호텔 피에드 수성,미등록,3급,0507-1448-2902,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 알파시티1로4길 18,35.839349,128.688500
3,대구 그랜드호텔,별점 4.25,4급,053-742-0001,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 동대구로 305,35.857751,128.625358
4,호텔라온제나,별점 4.39,4성급,053-718-7000,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 범어천로 73,35.861821,128.621766
5,진밭골야영장,미등록,"캠핑,야영장",053-666-3477,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 진밭길 112,35.817108,128.655764
6,ACT 관광호텔,미등록,3성급,0507-1384-0000,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 알파시티1로32길 28-17,35.836909,128.682084
7,2월호텔 라이온즈파크점,미등록,3급,053-792-2022,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 알파시티1로4길 19-7,35.839349,128.688500
8,아리아나호텔,별점 4.38,3성급,0507-1428-7803,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 동대구로 27,35.844965,128.625647
9,예향아트하우스,미등록,펜션,010-3537-0719,https://search.pstatic.net/common/?autoRotate=...,대구 수성구 고모로18길 22,35.850541,128.687437


In [85]:
import folium

# 수성구의 위도와 경도
latitude = 35.852622
longitude = 128.662269

#지도를 생성(위도와 경도 기준)
# 지도를 생성 (위도와 경도 기준)
map_suseong = folium.Map(location=[latitude, longitude], zoom_start=14)

# 마커 추가
for store_name, rating, adress, phone_num, cetegory, image,lat, lng in zip(loddf['store_name'],loddf['rating'],loddf['address'],loddf['phone_num'],loddf['category'],
    loddf['img'],loddf['latitude'],loddf['longitude']):
    popup_html = f"""
    <img src="{image}" width="200px" />
    <b>가게이름:</b> {store_name}<br>
    <b>평점:</b>{rating}<br>
    <b>카테고리:</b> {category}<br>
    <b>주소:</b> {adress}<br>
    <b>전화번호:</b>{phone_num}<br>
    
    """
    
    folium.Marker(
        location=[lat, lng],
        popup=folium.Popup(popup_html, max_width=200),
        icon=folium.Icon(color='green', icon="home"),
        ).add_to(map_suseong)

# 지도를 HTML 파일로 저장
map_suseong.save("suseong_lodge.html")
map_suseong